In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

In [74]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None, max_feat = 10):
    
    #print("Actual value ", test_y)
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=max_feat, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [7]:
def create_samples(shap_explainer, iterations, row, features, top = None):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        
        #if shap_type == "kernel":
        #    shap_explainer = shap.KernelExplainer(cls.predict, trainingsample)
        #elif shap_type == "tree":
        #    shap_explainer = shap.TreeExplainer(cls)
        #elif shap_type == "deep":
        #    shap_explainer = shap.DeepExplainer(cls, background)
        
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        if top != None:
            for i in range(top):
                feat = importances[i]
                if feat[2] > 0:
                    rel_feat.append(feat)

            rel_exp.append(rel_feat)
        else:
            rel_exp = exp
        
    return exp, rel_exp

In [8]:
def dispersal(weights, features):
    feat_len = len(features)
    #print(feat_len)
    weights_by_feat = []
       
    for i in list(range(feat_len)):
        feat_weight = []
        for iteration in weights:
            feat_weight.append(iteration[i])
        weights_by_feat.append(feat_weight)
        
    #for iteration in weights:
     #   for val in iteration:
      #      idx = iteration.index(val)
       #     print(idx)
        #    weights_by_feat[idx].append(val)
    
    dispersal = []
    dispersal_no_outlier = []
    
    for each in weights_by_feat:
        #print("Feature", weights_by_feat.index(each)+1)
        mean = np.mean(each)
        std_dev = np.std(each)
        var = std_dev**2
        
        if mean == 0:
            dispersal.append(0)
            dispersal_no_outlier.append(0)
        #print(each)
        else:
            #dispersal with outliers
            rel_var = var/abs(mean)
            dispersal.append(rel_var)
            
            #dispersal without outliers - remove anything with a z-score higher
            #than 3 (more than 3 standard deviations away from the mean)
            rem_outlier = []
            z_scores = stats.zscore(each)
            #print(z_scores)
            #print("New list:")
            for i in range(len(z_scores)):
                #print(each[i],":",z_scores[i])
                if -3 < z_scores[i] < 3:
                    rem_outlier.append(each[i])
                #print(rem_outlier)
            if rem_outlier != []:
                new_mean = np.mean(rem_outlier)
                if new_mean == 0:
                    dispersal_no_outlier.append(0)
                else:
                    new_std = np.std(rem_outlier)
                    new_var = new_std**2
                    new_rel_var = new_var/abs(new_mean)
                    dispersal_no_outlier.append(new_rel_var)
            else:
                dispersal_no_outlier.append(rel_var)
    #print(dispersal_no_outlier)
    return dispersal, dispersal_no_outlier

In [9]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = False
generate_lime = True
generate_kernel_shap = False
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [10]:
for dataset_name in datasets:

    dataset_manager = DatasetManager(dataset_name)

    for ii in range(n_iter):
        num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))

        for bucket in list(num_buckets):
            bucketID = bucket+1
            print ('Bucket', bucketID)

            #import everything needed to sort and predict
            pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

            predictor = joblib.load(pipeline_path)
            cls = joblib.load(cls_path)
            feature_combiner = joblib.load(feat_comb_path)
            bucketer = joblib.load(bucketer_path)

            #import data for bucket
            X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            with open (X_train_path, 'rb') as f:
                dt_train_bucket = pickle.load(f)
            with open (Y_train_path, 'rb') as f:
                train_y = pickle.load(f)
            with open (X_test_path, 'rb') as f:
                dt_test_bucket = pickle.load(f)
            with open (Y_test_path, 'rb') as f:
                test_y = pickle.load(f)

            #import previously defined samples
            tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            sample_instances = []

            with open (tn_path, 'rb') as f:
                tn_list = pickle.load(f)
            with open (tp_path, 'rb') as f:
                tp_list = pickle.load(f)
            #with open (fn_path, 'rb') as f:
            #    fn_list = pickle.load(f)
            #with open (fp_path, 'rb') as f:
            #    fp_list = pickle.load(f)

            #save results to a list
            sample_instances.append(tn_list)
            sample_instances.append(tp_list)
            #sample_instances.append(fn_list)
            #sample_instances.append(fp_list)

            print(cls)

Bucket 1
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5353160434296517, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.05959997729599409, max_delta_step=0, max_depth=22,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.8342285287391545, tree_method=None,
              validate_parameters=False, verbosity=None)


In [158]:
input_ = sample_instances[0][0]['input']
new_input = input_
row = new_input.index[new_input['Activity'] == 'A_ACCEPTED-COMPLETE']
new_input = new_input.replace({'10932.0': None})
#new_input = new_input.loc[row]['Activity']='A_CANCELLED-COMPLETE'
#row = input_[input_['Activity']=='A_ACCEPTED-COMPLETE']
#row['Activity']='A_CANCELLED-COMPLETE'
#row
#new_input = input_.append(row)

row = np.full(len(input_.columns), None)
loc = input_.columns.get_loc('Activity')
row[loc] = 'A_CANCELLED-COMPLETE'
loc = input_.columns.get_loc('Case ID')
case_id = input_.iloc[0, loc]
row[loc] = case_id
row = pd.Series(row, index = input_.columns)
new_input = input_.append(row, ignore_index = True)
new_input

,AMOUNT_REQ,Case ID,label,Activity,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,Complete Timestamp,case_length,prefix_nr,orig_case_id
0,65000.0,204712_30,regular,A_SUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.000000,0.000000,1.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:31.378,40,30,204712
1,65000.0,204712_30,regular,A_PARTLYSUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.010450,0.010450,2.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:32.005,40,30,204712
2,65000.0,204712_30,regular,W_Afhandelen leads-SCHEDULE,112.0,SCHEDULE,719.0,0.062167,0.072617,3.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:35.735,40,30,204712
3,65000.0,204712_30,regular,W_Afhandelen leads-START,112.0,START,669.0,1390.024383,1390.097000,4.0,1.0,0.0,11.0,874.0,2012-01-30 11:09:37.198,40,30,204712
4,65000.0,204712_30,regular,A_PREACCEPTED-COMPLETE,11202.0,COMPLETE,674.0,5.325433,1395.422433,5.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.724,40,30,204712
5,65000.0,204712_30,regular,W_Completeren aanvraag-SCHEDULE,11202.0,SCHEDULE,674.0,0.000733,1395.423167,6.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.768,40,30,204712
6,65000.0,204712_30,regular,W_Afhandelen leads-COMPLETE,11202.0,COMPLETE,674.0,0.018633,1395.441800,7.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:57.886,40,30,204712
7,65000.0,204712_30,regular,W_Completeren aanvraag-START,11009.0,START,1026.0,351.825867,1747.267667,8.0,1.0,0.0,17.0,851.0,2012-01-30 17:06:47.438,40,30,204712
8,65000.0,204712_30,regular,A_ACCEPTED-COMPLETE,11009.0,COMPLETE,1039.0,12.869117,1760.136783,9.0,1.0,0.0,17.0,852.0,2012-01-30 17:19:39.585,40,30,204712
9,65000.0,204712_30,regular,O_SELECTED-COMPLETE,11009.0,COMPLETE,1048.0,8.924133,1769.060917,10.0,1.0,0.0,17.0,852.0,2012-01-30 17:28:35.033,40,30,204712


In [21]:
input_

,AMOUNT_REQ,Case ID,label,Activity,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,Complete Timestamp,case_length,prefix_nr,orig_case_id
152221,65000.0,204712_30,regular,A_SUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.000000,0.000000,1.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:31.378,40,30,204712
152222,65000.0,204712_30,regular,A_PARTLYSUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.010450,0.010450,2.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:32.005,40,30,204712
152223,65000.0,204712_30,regular,W_Afhandelen leads-SCHEDULE,112.0,SCHEDULE,719.0,0.062167,0.072617,3.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:35.735,40,30,204712
152224,65000.0,204712_30,regular,W_Afhandelen leads-START,112.0,START,669.0,1390.024383,1390.097000,4.0,1.0,0.0,11.0,874.0,2012-01-30 11:09:37.198,40,30,204712
152225,65000.0,204712_30,regular,A_PREACCEPTED-COMPLETE,11202.0,COMPLETE,674.0,5.325433,1395.422433,5.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.724,40,30,204712
152226,65000.0,204712_30,regular,W_Completeren aanvraag-SCHEDULE,11202.0,SCHEDULE,674.0,0.000733,1395.423167,6.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.768,40,30,204712
152227,65000.0,204712_30,regular,W_Afhandelen leads-COMPLETE,11202.0,COMPLETE,674.0,0.018633,1395.441800,7.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:57.886,40,30,204712
152228,65000.0,204712_30,regular,W_Completeren aanvraag-START,11009.0,START,1026.0,351.825867,1747.267667,8.0,1.0,0.0,17.0,851.0,2012-01-30 17:06:47.438,40,30,204712
152229,65000.0,204712_30,regular,A_ACCEPTED-COMPLETE,11009.0,COMPLETE,1039.0,12.869117,1760.136783,9.0,1.0,0.0,17.0,852.0,2012-01-30 17:19:39.585,40,30,204712
152230,65000.0,204712_30,regular,O_SELECTED-COMPLETE,11009.0,COMPLETE,1048.0,8.924133,1769.060917,10.0,1.0,0.0,17.0,852.0,2012-01-30 17:28:35.033,40,30,204712


In [159]:
test_x_group= feature_combiner.fit_transform(input_) 
#test_x=np.transpose(test_x_group[0])
p1 = cls.predict_proba(test_x_group)[0][0]
print(p1)
test_x_group

0.5014116


array([[6.50000000e+04, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 0.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 5.00000000e+00, 2.00000000e+00,
        5.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [160]:
test_x_group= feature_combiner.fit_transform(new_input) 
#test_x=np.transpose(test_x_group[0])
p2 = cls.predict_proba(test_x_group)[0][0]
print(p2)
test_x_group

0.99475986


array([[6.50000000e+04, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 0.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 5.00000000e+00, 2.00000000e+00,
        5.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [15]:
lime_explanations = []
trainingdata = feature_combiner.fit_transform(dt_train_bucket);
class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata, feature_names = feature_combiner.get_feature_names(),
                                          class_names=class_names, discretize_continuous=True)
test_x_group= feature_combiner.fit_transform(input_) 
test_x=np.transpose(test_x_group[0])
for i in range(10):
    lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, input_, max_feat = 10)
    lime_explanations.append(lime_exp.as_list())

In [26]:
test_x_group= feature_combiner.fit_transform(new_input) 
test_x=np.transpose(test_x_group[0])
lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, input_, max_feat = 300)
lime_exp.as_list()

[('agg__Activity_A_CANCELLED-COMPLETE <= 0.00', 0.5369333379820667),
 ('agg__Activity_A_DECLINED-COMPLETE <= 0.00', 0.42585635454298215),
 ('agg__Activity_A_APPROVED-COMPLETE <= 0.00', -0.3090519324605798),
 ('agg__Activity_A_ACTIVATED-COMPLETE <= 0.00', -0.3005476212726858),
 ('agg__Activity_O_SENT_BACK-COMPLETE <= 0.00', -0.26168322554821705),
 ('agg__Resource_10138.0 <= 0.00', -0.2580254591484931),
 ('agg__Resource_10972.0 <= 0.00', -0.22820904225111435),
 ('agg__Resource_10933.0 <= 0.00', -0.18885938916410203),
 ('agg__Resource_10629.0 <= 0.00', -0.17596778587195),
 ('agg__Resource_11049.0 <= 0.00', -0.17020963318880838),
 ('agg__Resource_11001.0 <= 0.00', 0.16329264597391502),
 ('agg__Activity_O_ACCEPTED-COMPLETE <= 0.00', -0.15305419496256925),
 ('agg__Activity_W_Wijzigen contractgegevens-SCHEDULE <= 0.00',
  -0.14143886932358396),
 ('agg__Resource_10809.0 <= 0.00', -0.13995850317752273),
 ('agg__Resource_10789.0 <= 0.00', -0.1385408635848803),
 ('agg__Resource_10932.0 <= 0.00', -0.1171346314056444),
 ('agg__Resource_11019.0 <= 0.00', 0.09911026705711222),
 ('agg__Resource_11111.0 <= 0.00', -0.09505050036920572),
 ('agg__Resource_other <= 0.00', 0.08914243237820589),
 ('agg__Activity_W_Beoordelen fraude-SCHEDULE <= 0.00', -0.07731585236059804),
 ('agg__Resource_11003.0 <= 0.00', -0.0699586091106297),
 ('agg__Resource_11009.0 > 0.00', 0.0639312199748199),
 ('agg__Activity_W_Valideren aanvraag-SCHEDULE <= 0.00', -0.06324628356507288),
 ('agg__Resource_11002.0 <= 0.00', -0.061985297157529255),
 ('agg__Activity_W_Nabellen incomplete dossiers-SCHEDULE <= 0.00',
  -0.056643242057436045),
 ('agg__Activity_W_Beoordelen fraude-COMPLETE <= 0.00', -0.0539578173410765),
 ('agg__timesincecasestart_max > 14530.74', -0.05304806447446561),
 ('agg__Resource_10889.0 <= 0.00', -0.050736642725147996),
 ('agg__Activity_A_REGISTERED-COMPLETE <= 0.00', -0.04942516406276966),
 ('agg__Activity_W_Valideren aanvraag-START <= 0.00', -0.04901122013995745),
 ('agg__Activity_W_Completeren aanvraag-START <= 1.00', 0.048133186320819026),
 ('agg__Resource_10910.0 <= 0.00', 0.044466681763843934),
 ('agg__Resource_10931.0 <= 0.00', -0.0434242486448316),
 ('agg__Resource_10982.0 <= 0.00', 0.04305166181678632),
 ('agg__Resource_11202.0 > 0.00', -0.04255642203340059),
 ('agg__Resource_10939.0 <= 0.00', 0.04112774483707634),
 ('agg__Resource_10971.0 <= 0.00', -0.0404617919207696),
 ('agg__Resource_10914.0 <= 0.00', 0.03898662805731084),
 ('agg__Resource_11201.0 <= 0.00', 0.03676761076499432),
 ('agg__Resource_10880.0 <= 0.00', 0.035113048370981184),
 ('0.00 < agg__Activity_A_ACCEPTED-COMPLETE <= 1.00', 0.03428216564124577),
 ('agg__Resource_11200.0 <= 0.00', 0.032539220562013374),
 ('agg__Resource_11180.0 <= 0.00', 0.0322027667741626),
 ('agg__Resource_11181.0 <= 0.00', 0.031935788245901126),
 ('agg__Resource_11000.0 <= 0.00', -0.030790370486459048),
 ('agg__Resource_10779.0 <= 0.00', -0.030194357078108677),
 ('agg__Resource_10862.0 <= 0.00', -0.029759225627985317),
 ('agg__open_cases_min > 721.00', 0.029306852016828366),
 ('agg__Resource_10863.0 <= 0.00', 0.02908363480109153),
 ('634.00 < agg__timesincemidnight_min <= 709.00', 0.02763158433019209),
 ('agg__timesincelastevent_max > 9994.23', -0.026901214173363348),
 ('agg__Resource_11203.0 <= 0.00', -0.023248373239807156),
 ('agg__Activity_W_Completeren aanvraag-SCHEDULE <= 1.00',
  -0.022797456107130024),
 ('0.00 < agg__Activity_W_Afhandelen leads-START <= 1.00',
  -0.02237187978253513),
 ('0.00 < agg__Activity_W_Afhandelen leads-SCHEDULE <= 1.00',
  -0.02218965518468087),
 ('agg__Resource_11120.0 <= 0.00', -0.021285918809482076),
 ('agg__Resource_10912.0 <= 0.00', -0.020946351064472946),
 ('agg__Activity_O_CANCELLED-COMPLETE > 0.00', 0.020755678663576194),
 ('agg__Resource_11169.0 <= 0.00', -0.019773706593374542),
 ('agg__Resource_11119.0 <= 0.00', 0.019340304971392466),
 ('agg__Resource_11122.0 <= 0.00', -0.019083892551850206),
 ('agg__Resource_11259.0 <= 0.00', -0.017447874232070327),
 ('agg__Activity_W_Be

In [16]:
difference = p1-p2
difference

-0.03396207

In [17]:
tree_explainer = shap.TreeExplainer(cls)
test_x_group= feature_combiner.fit_transform(input_) 
test_x=np.transpose(test_x_group[0])
feat_list = feature_combiner.get_feature_names()
exp, rel_exp = create_samples(tree_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)


In [18]:
features = []
for each in rel_exp:
    for feature in each:
        features.append(feature[0])
        
len(features)

import collections
counter = collections.Counter(features)
counter

Counter({'agg__Resource_10932.0': 10,
         'static__AMOUNT_REQ': 10,
         'agg__timesincelastevent_max': 10,
         'agg__weekday_max': 10,
         'agg__Resource_11009.0': 10,
         'agg__timesincecasestart_max': 10,
         'agg__Activity_W_Afhandelen leads-SCHEDULE': 10,
         'agg__timesincemidnight_min': 10,
         'agg__timesincelastevent_sum': 10,
         'agg__open_cases_mean': 10})

In [19]:
l_features = []
for each in lime_explanations:
    for feature in each:
        l_features.append(feature[0])
        
len(l_features)

import collections
counter = collections.Counter(l_features)
counter

Counter({'agg__Activity_A_CANCELLED-COMPLETE <= 0.00': 10,
         'agg__Activity_A_DECLINED-COMPLETE <= 0.00': 10,
         'agg__Activity_A_APPROVED-COMPLETE <= 0.00': 10,
         'agg__Activity_A_ACTIVATED-COMPLETE <= 0.00': 10,
         'agg__Activity_O_SENT_BACK-COMPLETE <= 0.00': 10,
         'agg__Resource_10138.0 <= 0.00': 10,
         'agg__Resource_10972.0 <= 0.00': 10,
         'agg__Activity_W_Completeren aanvraag-SCHEDULE <= 1.00': 1,
         'agg__Resource_10629.0 <= 0.00': 3,
         'agg__Resource_11049.0 <= 0.00': 3,
         'agg__Resource_10779.0 <= 0.00': 3,
         'agg__Activity_O_ACCEPTED-COMPLETE <= 0.00': 5,
         'agg__Activity_W_Beoordelen fraude-SCHEDULE <= 0.00': 1,
         'agg__Resource_other <= 0.00': 1,
         'agg__Resource_10914.0 <= 0.00': 1,
         'agg__Activity_W_Wijzigen contractgegevens-SCHEDULE <= 0.00': 4,
         'agg__Resource_10809.0 <= 0.00': 2,
         'agg__Resource_11079.0 <= 0.00': 1,
         'agg__Resource_11001.0 <= 0

In [100]:
case_id_col = dataset_manager.case_id_col
static_cat_cols = dataset_manager.static_cat_cols,
static_num_cols = dataset_manager.static_num_cols, 
dynamic_cat_cols = dataset_manager.dynamic_cat_cols,
dynamic_num_cols = dataset_manager.dynamic_num_cols,

In [99]:
dynamic_num_cols = tuple(num_cols[0])
dynamic_cat_cols = tuple(dynamic_cat_cols[0])

In [101]:
dynamic_cat_cols

(['Activity', 'Resource'],)

In [102]:
X = new_input
X

,AMOUNT_REQ,Case ID,label,Activity,Resource,lifecycle:transition,timesincemidnight,timesincelastevent,timesincecasestart,event_nr,month,weekday,hour,open_cases,Complete Timestamp,case_length,prefix_nr,orig_case_id
152221,65000.0,204712_30,regular,A_SUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.000000,0.000000,1.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:31.378,40,30,204712
152222,65000.0,204712_30,regular,A_PARTLYSUBMITTED-COMPLETE,112.0,COMPLETE,719.0,0.010450,0.010450,2.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:32.005,40,30,204712
152223,65000.0,204712_30,regular,W_Afhandelen leads-SCHEDULE,112.0,SCHEDULE,719.0,0.062167,0.072617,3.0,1.0,6.0,11.0,840.0,2012-01-29 11:59:35.735,40,30,204712
152224,65000.0,204712_30,regular,W_Afhandelen leads-START,112.0,START,669.0,1390.024383,1390.097000,4.0,1.0,0.0,11.0,874.0,2012-01-30 11:09:37.198,40,30,204712
152225,65000.0,204712_30,regular,A_PREACCEPTED-COMPLETE,11202.0,COMPLETE,674.0,5.325433,1395.422433,5.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.724,40,30,204712
152226,65000.0,204712_30,regular,W_Completeren aanvraag-SCHEDULE,11202.0,SCHEDULE,674.0,0.000733,1395.423167,6.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:56.768,40,30,204712
152227,65000.0,204712_30,regular,W_Afhandelen leads-COMPLETE,11202.0,COMPLETE,674.0,0.018633,1395.441800,7.0,1.0,0.0,11.0,875.0,2012-01-30 11:14:57.886,40,30,204712
152228,65000.0,204712_30,regular,W_Completeren aanvraag-START,11009.0,START,1026.0,351.825867,1747.267667,8.0,1.0,0.0,17.0,851.0,2012-01-30 17:06:47.438,40,30,204712
152229,65000.0,204712_30,regular,A_ACCEPTED-COMPLETE,11009.0,COMPLETE,1039.0,12.869117,1760.136783,9.0,1.0,0.0,17.0,852.0,2012-01-30 17:19:39.585,40,30,204712
152230,65000.0,204712_30,regular,O_SELECTED-COMPLETE,11009.0,COMPLETE,1048.0,8.924133,1769.060917,10.0,1.0,0.0,17.0,852.0,2012-01-30 17:28:35.033,40,30,204712


In [117]:
if len(dynamic_num_cols) > 0:
    dt_numeric = X.groupby(by=[case_id_col])
    dt_numeric = dt_numeric[dynamic_num_cols[0]].agg(['mean', 'max', 'min', 'sum', 'std'])
dt_numeric

hour                                weekday                  \
                mean   max   min    sum       std      mean  max  min   sum   
Case ID                                                                       
204712_30  14.866667  17.0  11.0  446.0  2.861557  0.933333  6.0  0.0  28.0   

                    ... event_nr                              open_cases  \
               std  ...     mean   max  min    sum       std        mean   
Case ID             ...                                                    
204712_30  1.99885  ...     15.5  30.0  1.0  465.0  8.803408  856.933333   

                                             
             max    min      sum        std  
Case ID                                      
204712_30  880.0  840.0  25708.0  11.837793  

[1 rows x 40 columns]

In [118]:
dt_transformed = pd.get_dummies(X[dynamic_cat_cols[0]])
dt_transformed[case_id_col] = X[case_id_col]
dt_transformed = dt_transformed.groupby(case_id_col).sum()
dt_transformed = pd.concat([dt_transformed, dt_numeric], axis=1)
dt_transformed

,Activity_A_ACCEPTED-COMPLETE,Activity_A_FINALIZED-COMPLETE,Activity_A_PARTLYSUBMITTED-COMPLETE,Activity_A_PREACCEPTED-COMPLETE,Activity_A_SUBMITTED-COMPLETE,Activity_O_CANCELLED-COMPLETE,Activity_O_CREATED-COMPLETE,Activity_O_SELECTED-COMPLETE,Activity_O_SENT-COMPLETE,Activity_W_Afhandelen leads-COMPLETE,...,"(event_nr, mean)","(event_nr, max)","(event_nr, min)","(event_nr, sum)","(event_nr, std)","(open_cases, mean)","(open_cases, max)","(open_cases, min)","(open_cases, sum)","(open_cases, std)"
Case ID,,,,,,,,,,,,,,,,,,,,,
204712_30,1,1,1,1,1,1,2,2,2,1,...,15.5,30.0,1.0,465.0,8.803408,856.933333,880.0,840.0,25708.0,11.837793


In [119]:
dt_transformed.columns

Index([            'Activity_A_ACCEPTED-COMPLETE',
                  'Activity_A_FINALIZED-COMPLETE',
            'Activity_A_PARTLYSUBMITTED-COMPLETE',
                'Activity_A_PREACCEPTED-COMPLETE',
                  'Activity_A_SUBMITTED-COMPLETE',
                  'Activity_O_CANCELLED-COMPLETE',
                    'Activity_O_CREATED-COMPLETE',
                   'Activity_O_SELECTED-COMPLETE',
                       'Activity_O_SENT-COMPLETE',
           'Activity_W_Afhandelen leads-COMPLETE',
           'Activity_W_Afhandelen leads-SCHEDULE',
              'Activity_W_Afhandelen leads-START',
       'Activity_W_Completeren aanvraag-COMPLETE',
       'Activity_W_Completeren aanvraag-SCHEDULE',
          'Activity_W_Completeren aanvraag-START',
          'Activity_W_Nabellen offertes-COMPLETE',
          'Activity_W_Nabellen offertes-SCHEDULE',
             'Activity_W_Nabellen offertes-START',
                               'Resource_11009.0',
                               

In [120]:
dt_transformed = dt_transformed.fillna(0)
dt_transformed

,Activity_A_ACCEPTED-COMPLETE,Activity_A_FINALIZED-COMPLETE,Activity_A_PARTLYSUBMITTED-COMPLETE,Activity_A_PREACCEPTED-COMPLETE,Activity_A_SUBMITTED-COMPLETE,Activity_O_CANCELLED-COMPLETE,Activity_O_CREATED-COMPLETE,Activity_O_SELECTED-COMPLETE,Activity_O_SENT-COMPLETE,Activity_W_Afhandelen leads-COMPLETE,...,"(event_nr, mean)","(event_nr, max)","(event_nr, min)","(event_nr, sum)","(event_nr, std)","(open_cases, mean)","(open_cases, max)","(open_cases, min)","(open_cases, sum)","(open_cases, std)"
Case ID,,,,,,,,,,,,,,,,,,,,,
204712_30,1,1,1,1,1,1,2,2,2,1,...,15.5,30.0,1.0,465.0,8.803408,856.933333,880.0,840.0,25708.0,11.837793


In [121]:
dt_first = X.groupby(case_id_col).first()
dt_transformed_2 = dt_first[static_num_cols[0]]
dt_transformed_2

,AMOUNT_REQ
Case ID,
204712_30,65000.0


In [122]:
#dt_cat = pd.get_dummies(dt_first[static_cat_cols[0]])
dt_transformed = pd.concat([dt_transformed, dt_transformed_2], axis=1)
dt_transformed

,Activity_A_ACCEPTED-COMPLETE,Activity_A_FINALIZED-COMPLETE,Activity_A_PARTLYSUBMITTED-COMPLETE,Activity_A_PREACCEPTED-COMPLETE,Activity_A_SUBMITTED-COMPLETE,Activity_O_CANCELLED-COMPLETE,Activity_O_CREATED-COMPLETE,Activity_O_SELECTED-COMPLETE,Activity_O_SENT-COMPLETE,Activity_W_Afhandelen leads-COMPLETE,...,"(event_nr, max)","(event_nr, min)","(event_nr, sum)","(event_nr, std)","(open_cases, mean)","(open_cases, max)","(open_cases, min)","(open_cases, sum)","(open_cases, std)",AMOUNT_REQ
Case ID,,,,,,,,,,,,,,,,,,,,,
204712_30,1,1,1,1,1,1,2,2,2,1,...,30.0,1.0,465.0,8.803408,856.933333,880.0,840.0,25708.0,11.837793,65000.0


In [129]:
test_x = feature_combiner.transform(X)
len(test_x[0])

134

In [ ]:
cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}



In [69]:
test_x

array([[6.50000000e+04, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 0.00000000e+00, 2.00000000e+00,
        2.00000000e+00, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 5.00000000e+00, 2.00000000e+00,
        5.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [65]:
dataset_manager.dynamic_cat_cols

['Activity', 'Resource']